In [28]:
from google.colab import drive
import os

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transfroms
import numpy as np


from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from PIL import Image
from torchvision.datasets import ImageFolder

In [119]:
#data load
BATCH_SIZE = 100
dataset = ImageFolder(root='/content/drive/MyDrive/ML/data/train-50', transform=transforms.ToTensor())
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [120]:
class myCNN(nn.Module):
    def __init__(self):
        super(myCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, padding='same')
        self.conv2 = nn.Conv2d(32, 64, 3, 1, padding='same')
        self.conv3 = nn.Conv2d(64, 128, 3, 1, padding='same')
        self.dropout = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(6272, 3000)    # 7 * 7 * 128 = 6272
        self.fc2 = nn.Linear(3000, 1000)
        self.fc3 = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.conv1(x) # 28 * 28 * 32
        x = F.relu(x)
        x = F.max_pool2d(x, 2) # 14 * 14 * 32
        x = self.conv2(x) # 14 * 14 * 64
        x = F.relu(x)
        x = F.max_pool2d(x, 2) # 7 * 7 * 64
        x = self.dropout(x)
        x = self.conv3(x) # 7 * 7 * 128
        x = F.relu(x)
        x = self.dropout(x)
        x = torch.flatten(x, 1) # = 6272
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

model = myCNN().to(DEVICE) # CNN instance 생성

In [121]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=1):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, output, target):
        ce_loss = F.cross_entropy(output, target, reduction='none')
        pt = torch.exp(- ce_loss)
        focal_loss = self.alpha * (1 - pt)**self.gamma * ce_loss
        return torch.mean(focal_loss)

criterion = FocalLoss(gamma=2, alpha=1).to(DEVICE)

In [122]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [123]:
epochs = 50

for epoch in range(epochs) :
    avg_loss = 0
    total_batch = len(train_loader)
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        #train epoch마다 loss 및 accuracy 확인 필요
        avg_loss += loss / total_batch
    print("Epoch: ", "%d\t" % (epoch+1), "loss =", "{:.9f}".format(avg_loss))
torch.save(model, '/content/drive/MyDrive/ML/checkpoint/20192940_JungyuKang.pth')

Epoch:  1	 loss = 1.839734316
Epoch:  2	 loss = 1.414924026
Epoch:  3	 loss = 0.858437419
Epoch:  4	 loss = 0.534354389
Epoch:  5	 loss = 0.320421547
Epoch:  6	 loss = 0.315510690
Epoch:  7	 loss = 0.204686329
Epoch:  8	 loss = 0.148678198
Epoch:  9	 loss = 0.090347111
Epoch:  10	 loss = 0.057780281
Epoch:  11	 loss = 0.033604153
Epoch:  12	 loss = 0.033544991
Epoch:  13	 loss = 0.014295530
Epoch:  14	 loss = 0.016328942
Epoch:  15	 loss = 0.011074693
Epoch:  16	 loss = 0.008835700
Epoch:  17	 loss = 0.002953822
Epoch:  18	 loss = 0.005099068
Epoch:  19	 loss = 0.002611792
Epoch:  20	 loss = 0.001721430
Epoch:  21	 loss = 0.001467180
Epoch:  22	 loss = 0.002612345
Epoch:  23	 loss = 0.004102447
Epoch:  24	 loss = 0.001056120
Epoch:  25	 loss = 0.006628218
Epoch:  26	 loss = 0.004587047
Epoch:  27	 loss = 0.000557282
Epoch:  28	 loss = 0.009286342
Epoch:  29	 loss = 0.011845455
Epoch:  30	 loss = 0.015213267
Epoch:  31	 loss = 0.006862226
Epoch:  32	 loss = 0.025964450
Epoch:  33	 loss 

In [124]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import dataloader
from collections import OrderedDict
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.datasets import ImageFolder

import easydict

args = easydict.EasyDict({
    "path":'/content/drive/MyDrive/ML/checkpoint/20192940_JungyuKang.pth',
    "batch":100
})
device = "cuda" if torch.cuda.is_available() else "cpu"

model = torch.load(args.path, map_location=device)
model = model.to(device)

#test
correct = 0
model = model.eval()


with torch.no_grad():
    for image, label in test_loader:
        image = image.to(device)
        label = label.to(device)
        output = model(image)
        prediction = output.max(1, keepdim=True)[1]
        correct += prediction.eq(label.view_as(prediction)).sum().item()
test_accuracy = 100. * correct/len(test_loader.dataset)

print("Test Accuracy: {:.2f}".format(test_accuracy))

Test Accuracy: 92.00
